In [1]:
import pandas as pd
import os
import mygene
from tqdm.notebook import tqdm

In [2]:
mg = mygene.MyGeneInfo()

# Asthma

In [3]:
asthma_data = pd.read_csv("processed_data/gwas_gene_pvals/asthma/PEGASUS_asthma_gwas_data.csv")
asthma_data

,Chr,Gene,nSNPs,Start,Stop,Test-Stat,Pvalue,Error,Best-SNP,SNP-pvalue
0,1,A3GALT2,68,33772366.0,33786699.0,44.811425,0.574424,1.292712e-14,rs10914692,0.009231
1,1,AADACL3,94,12776117.0,12788726.0,83.344493,0.451577,7.943619e-15,rs3000905,0.052147
2,1,AADACL4,54,12704565.0,12727097.0,39.493893,0.656680,9.349860e-15,rs11121955,0.012345
3,1,ABCA4,261,94458393.0,94586705.0,210.388204,0.688397,3.012240e-15,rs11165076,0.025043
4,1,ABCB10,60,229652328.0,229694442.0,41.712991,0.483655,8.694164e-15,rs12734075,0.089851
...,...,...,...,...,...,...,...,...,...,...
17656,22,ZNF280A,51,22868060.0,22874613.0,51.315894,0.381351,1.428313e-14,rs2330191,0.065178
17657,22,ZNF280B,51,22838771.0,22863505.0,44.383404,0.457183,9.159330e-15,rs2266984,0.067977
17658,22,ZNF70,58,24083771.0,24093279.0,56.767506,0.415667,6.594791e-15,rs738791,0.070605
17659,22,ZNF74,54,20748479.0,20762752.0,198.180693,0.001879,6.312862e-14,rs1153427,0.000161


In [4]:
gids = list(asthma_data["Gene"])
gene_mapping = mg.querymany(gids, scopes=["symbol", "alias"], species="human", returnall=True)
gene_mapping["out"][:10]

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-17661...done.
Finished.
1001 input query terms found dup hits:
	[('ADC', 2), ('AGT', 2), ('AIDA', 2), ('AK3L1', 2), ('APITD1', 2), ('APOA2', 2), ('ARF1', 2), ('ARNT
508 input query terms found no hit:
	['AMY1A-1', 'AMY1A-3', 'AMY1B-1', 'AMY1B-3', 'AMY1C-1', 'AMY1C-3', 'FLJ13137', 'FLJ45717', 'GPR89C-2


[{'query': 'A3GALT2',
  '_id': '127550',
  '_score': 18.73485,
  'entrezgene': '127550',
  'name': 'alpha 1,3-galactosyltransferase 2',
  'symbol': 'A3GALT2',
  'taxid': 9606},
 {'query': 'AADACL3',
  '_id': '126767',
  '_score': 19.469177,
  'entrezgene': '126767',
  'name': 'arylacetamide deacetylase like 3',
  'symbol': 'AADACL3',
  'taxid': 9606},
 {'query': 'AADACL4',
  '_id': '343066',
  '_score': 18.745165,
  'entrezgene': '343066',
  'name': 'arylacetamide deacetylase like 4',
  'symbol': 'AADACL4',
  'taxid': 9606},
 {'query': 'ABCA4',
  '_id': '24',
  '_score': 18.067741,
  'entrezgene': '24',
  'name': 'ATP binding cassette subfamily A member 4',
  'symbol': 'ABCA4',
  'taxid': 9606},
 {'query': 'ABCB10',
  '_id': '23456',
  '_score': 17.55544,
  'entrezgene': '23456',
  'name': 'ATP binding cassette subfamily B member 10',
  'symbol': 'ABCB10',
  'taxid': 9606},
 {'query': 'ABCD3',
  '_id': '5825',
  '_score': 18.064056,
  'entrezgene': '5825',
  'name': 'ATP binding casset

In [5]:
# Try again removing symbols from missing genes
refined_query = [g.replace("-" , "").replace(".", "") for g in gene_mapping["missing"]]
secondary_gene_mapping = mg.querymany(refined_query, scopes=["symbol", "alias"], species="human", returnall=True)
secondary_gene_mapping["out"][:4]

querying 1-508...done.
Finished.
1 input query terms found dup hits:
	[('PC3', 5)]
501 input query terms found no hit:
	['AMY1A1', 'AMY1A3', 'AMY1B1', 'AMY1B3', 'AMY1C1', 'AMY1C3', 'FLJ13137', 'FLJ45717', 'GPR89C2', 'HIS


[{'query': 'AMY1A1', 'notfound': True},
 {'query': 'AMY1A3', 'notfound': True},
 {'query': 'AMY1B1', 'notfound': True},
 {'query': 'AMY1B3', 'notfound': True}]

In [10]:
ncbi_ids = []
for gn in tqdm(gids):
    res = [e for e in gene_mapping["out"] if e["query"]==gn]
    res = [r for r in res if "_score" in r.keys() and "entrezgene" in r.keys()]
    if len(res)<1:
        ncbi_ids.append(None)
        continue
        
    res = sorted(res, key=lambda x: x["_score"], reverse=True)
    ncbi_ids.append(res[0]["entrezgene"])
print("None values ", len([i for i in ncbi_ids if i is None]))
len(ncbi_ids)

  0%|          | 0/17661 [00:00<?, ?it/s]

None values  509


17661

In [11]:
asthma_data["NCBI_id"] = ncbi_ids
asthma_data = asthma_data[~asthma_data["NCBI_id"].isna()]
asthma_data

,Chr,Gene,nSNPs,Start,Stop,Test-Stat,Pvalue,Error,Best-SNP,SNP-pvalue,NCBI_id
0,1,A3GALT2,68,33772366.0,33786699.0,44.811425,0.574424,1.292712e-14,rs10914692,0.009231,127550
1,1,AADACL3,94,12776117.0,12788726.0,83.344493,0.451577,7.943619e-15,rs3000905,0.052147,126767
2,1,AADACL4,54,12704565.0,12727097.0,39.493893,0.656680,9.349860e-15,rs11121955,0.012345,343066
3,1,ABCA4,261,94458393.0,94586705.0,210.388204,0.688397,3.012240e-15,rs11165076,0.025043,24
4,1,ABCB10,60,229652328.0,229694442.0,41.712991,0.483655,8.694164e-15,rs12734075,0.089851,23456
...,...,...,...,...,...,...,...,...,...,...,...
17656,22,ZNF280A,51,22868060.0,22874613.0,51.315894,0.381351,1.428313e-14,rs2330191,0.065178,129025
17657,22,ZNF280B,51,22838771.0,22863505.0,44.383404,0.457183,9.159330e-15,rs2266984,0.067977,140883
17658,22,ZNF70,58,24083771.0,24093279.0,56.767506,0.415667,6.594791e-15,rs738791,0.070605,7621
17659,22,ZNF74,54,20748479.0,20762752.0,198.180693,0.001879,6.312862e-14,rs1153427,0.000161,7625


In [12]:
asthma_data.to_csv("processed_data/gwas_gene_pvals/asthma/filtered_ncbi_PEGASUS_asthma_gwas_data.csv", index=False)

# Autism

In [3]:
autism_data = pd.read_csv("processed_data/gwas_gene_pvals/autism/PEGASUS_autism_gwas_data.csv")
autism_data

,Chr,Gene,nSNPs,Start,Stop,Test-Stat,Pvalue,Error,Best-SNP,SNP-pvalue
0,1,A3GALT2,89,33772366.0,33786699.0,137.053070,0.185235,4.569034e-14,rs4653044,0.001152
1,1,AADACL3,132,12776117.0,12788726.0,32.016457,0.970413,1.839305e-14,rs3010873,0.265100
2,1,AADACL4,110,12704565.0,12727097.0,84.549660,0.588825,6.345503e-15,rs6699993,0.006778
3,1,ABCA4,342,94458393.0,94586705.0,370.418540,0.341536,5.743797e-15,rs3120133,0.025470
4,1,ABCB10,86,229652328.0,229694442.0,142.749985,0.153780,3.450117e-14,rs10916485,0.036440
...,...,...,...,...,...,...,...,...,...,...
17768,22,ZNF280A,14,22868060.0,22874613.0,13.958949,0.318221,1.988166e-11,rs3747098,0.195700
17769,22,ZNF280B,21,22838771.0,22863505.0,17.537348,0.431712,2.477566e-14,rs3747098,0.195700
17770,22,ZNF70,75,24083771.0,24093279.0,86.686562,0.304061,1.291541e-14,rs6003875,0.018310
17771,22,ZNF74,82,20748479.0,20762752.0,53.933570,0.688966,1.232943e-14,rs8748,0.065030


In [8]:
gids = list(autism_data["Gene"])
gene_mapping = mg.querymany(gids, scopes=["symbol", "alias"], species="human", returnall=True)
gene_mapping["out"][:4]

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-17773...done.
Finished.
998 input query terms found dup hits:
	[('ADC', 2), ('AGT', 2), ('AIDA', 2), ('AK3L1', 2), ('APITD1', 2), ('APOA2', 2), ('ARF1', 2), ('ARNT
562 input query terms found no hit:
	['AMY1A-1', 'AMY1A-3', 'AMY1B-1', 'AMY1B-3', 'AMY1C-1', 'AMY1C-3', 'FLJ13137', 'FLJ45717', 'GPR89C-2


[{'query': 'A3GALT2',
  '_id': '127550',
  '_score': 18.802341,
  'entrezgene': '127550',
  'name': 'alpha 1,3-galactosyltransferase 2',
  'symbol': 'A3GALT2',
  'taxid': 9606},
 {'query': 'AADACL3',
  '_id': '126767',
  '_score': 19.529512,
  'entrezgene': '126767',
  'name': 'arylacetamide deacetylase like 3',
  'symbol': 'AADACL3',
  'taxid': 9606},
 {'query': 'AADACL4',
  '_id': '343066',
  '_score': 18.81738,
  'entrezgene': '343066',
  'name': 'arylacetamide deacetylase like 4',
  'symbol': 'AADACL4',
  'taxid': 9606},
 {'query': 'ABCA4',
  '_id': '24',
  '_score': 18.21474,
  'entrezgene': '24',
  'name': 'ATP binding cassette subfamily A member 4',
  'symbol': 'ABCA4',
  'taxid': 9606}]

In [9]:
ncbi_ids = []
for gn in tqdm(gids):
    res = [e for e in gene_mapping["out"] if e["query"]==gn]
    res = [r for r in res if "_score" in r.keys() and "entrezgene" in r.keys()]
    if len(res)<1:
        ncbi_ids.append(None)
        continue
#     elif len(res)>1:
#         break
        
    res = sorted(res, key=lambda x: x["_score"], reverse=True)
    ncbi_ids.append(res[0]["entrezgene"])
print("None values ", len([i for i in ncbi_ids if i is None]))
len(ncbi_ids)

  0%|          | 0/17773 [00:00<?, ?it/s]

None values  563


17773

In [10]:
autism_data["NCBI_id"] = ncbi_ids
autism_data = autism_data[~autism_data["NCBI_id"].isna()]
autism_data

,Chr,Gene,nSNPs,Start,Stop,Test-Stat,Pvalue,Error,Best-SNP,SNP-pvalue,NCBI_id
0,1,A3GALT2,89,33772366.0,33786699.0,137.053070,0.185235,4.569034e-14,rs4653044,0.001152,127550
1,1,AADACL3,132,12776117.0,12788726.0,32.016457,0.970413,1.839305e-14,rs3010873,0.265100,126767
2,1,AADACL4,110,12704565.0,12727097.0,84.549660,0.588825,6.345503e-15,rs6699993,0.006778,343066
3,1,ABCA4,342,94458393.0,94586705.0,370.418540,0.341536,5.743797e-15,rs3120133,0.025470,24
4,1,ABCB10,86,229652328.0,229694442.0,142.749985,0.153780,3.450117e-14,rs10916485,0.036440,23456
...,...,...,...,...,...,...,...,...,...,...,...
17768,22,ZNF280A,14,22868060.0,22874613.0,13.958949,0.318221,1.988166e-11,rs3747098,0.195700,129025
17769,22,ZNF280B,21,22838771.0,22863505.0,17.537348,0.431712,2.477566e-14,rs3747098,0.195700,140883
17770,22,ZNF70,75,24083771.0,24093279.0,86.686562,0.304061,1.291541e-14,rs6003875,0.018310,7621
17771,22,ZNF74,82,20748479.0,20762752.0,53.933570,0.688966,1.232943e-14,rs8748,0.065030,7625


In [11]:
autism_data.to_csv("processed_data/gwas_gene_pvals/autism/filtered_ncbi_PEGASUS_autism_gwas_data.csv", index=False)

# Schizophrenia

In [18]:
sch_data = pd.read_csv("processed_data/gwas_gene_pvals/schizophrenia/PEGASUS_schizophrenia_gwas_data.csv")
sch_data

,Chr,Gene,nSNPs,Start,Stop,Test-Stat,Pvalue,Error,Best-SNP,SNP-pvalue
0,1,A3GALT2,89,33772366.0,33786699.0,242.644950,0.039346,1.017755e-13,rs12034407,1.400000e-03
1,1,AADACL3,133,12776117.0,12788726.0,181.875486,0.213243,2.296784e-14,rs3000870,6.200000e-03
2,1,AADACL4,115,12704565.0,12727097.0,108.047041,0.431211,4.167978e-15,rs3000870,6.200000e-03
3,1,ABCA4,346,94458393.0,94586705.0,449.555425,0.162139,1.278153e-14,rs581244,2.000000e-03
4,1,ABCB10,85,229652328.0,229694442.0,29.481566,0.843688,1.592276e-14,rs348326,8.830000e-02
...,...,...,...,...,...,...,...,...,...,...
17484,22,ZDHHC8,123,20119364.0,20133974.0,859.926651,0.000004,1.008769e-13,rs8137258,9.870000e-07
17485,22,ZMAT5,146,30126944.0,30162969.0,620.945084,0.003213,7.477507e-14,rs737932,1.000000e-03
17486,22,ZNF70,76,24083771.0,24093279.0,133.184031,0.095392,2.946103e-14,rs17629631,6.700000e-03
17487,22,ZNF74,82,20748479.0,20762752.0,64.069484,0.576769,6.663133e-15,rs5749316,2.820000e-02


In [19]:
gids = list(sch_data["Gene"])
gene_mapping = mg.querymany(gids, scopes=["symbol", "alias"], species="human", returnall=True)
gene_mapping["out"][:4]

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-17489...done.
Finished.
977 input query terms found dup hits:
	[('ADC', 2), ('AGT', 2), ('AIDA', 2), ('AK3L1', 2), ('APITD1', 2), ('APOA2', 2), ('ARF1', 2), ('ARNT
524 input query terms found no hit:
	['AMY1A-1', 'AMY1A-3', 'AMY1B-1', 'AMY1B-3', 'AMY1C-1', 'AMY1C-3', 'FLJ13137', 'FLJ45717', 'GPR89C-2


[{'query': 'A3GALT2',
  '_id': '127550',
  '_score': 92.45881,
  'entrezgene': '127550',
  'name': 'alpha 1,3-galactosyltransferase 2',
  'symbol': 'A3GALT2',
  'taxid': 9606},
 {'query': 'AADACL3',
  '_id': '126767',
  '_score': 96.26664,
  'entrezgene': '126767',
  'name': 'arylacetamide deacetylase like 3',
  'symbol': 'AADACL3',
  'taxid': 9606},
 {'query': 'AADACL4',
  '_id': '343066',
  '_score': 93.129395,
  'entrezgene': '343066',
  'name': 'arylacetamide deacetylase like 4',
  'symbol': 'AADACL4',
  'taxid': 9606},
 {'query': 'ABCA4',
  '_id': '24',
  '_score': 89.94049,
  'entrezgene': '24',
  'name': 'ATP binding cassette subfamily A member 4',
  'symbol': 'ABCA4',
  'taxid': 9606}]

In [20]:
ncbi_ids = []
for gn in tqdm(gids):
    res = [e for e in gene_mapping["out"] if e["query"]==gn]
    res = [r for r in res if "_score" in r.keys() and "entrezgene" in r.keys()]
    if len(res)<1:
        ncbi_ids.append(None)
        continue
    res = sorted(res, key=lambda x: x["_score"], reverse=True)
    ncbi_ids.append(res[0]["entrezgene"])
print("None values ", len([i for i in ncbi_ids if i is None]))
len(ncbi_ids)

  0%|          | 0/17489 [00:00<?, ?it/s]

None values  525


17489

In [21]:
sch_data["NCBI_id"] = ncbi_ids
sch_data = sch_data[~sch_data["NCBI_id"].isna()]
sch_data

,Chr,Gene,nSNPs,Start,Stop,Test-Stat,Pvalue,Error,Best-SNP,SNP-pvalue,NCBI_id
0,1,A3GALT2,89,33772366.0,33786699.0,242.644950,0.039346,1.017755e-13,rs12034407,1.400000e-03,127550
1,1,AADACL3,133,12776117.0,12788726.0,181.875486,0.213243,2.296784e-14,rs3000870,6.200000e-03,126767
2,1,AADACL4,115,12704565.0,12727097.0,108.047041,0.431211,4.167978e-15,rs3000870,6.200000e-03,343066
3,1,ABCA4,346,94458393.0,94586705.0,449.555425,0.162139,1.278153e-14,rs581244,2.000000e-03,24
4,1,ABCB10,85,229652328.0,229694442.0,29.481566,0.843688,1.592276e-14,rs348326,8.830000e-02,23456
...,...,...,...,...,...,...,...,...,...,...,...
17484,22,ZDHHC8,123,20119364.0,20133974.0,859.926651,0.000004,1.008769e-13,rs8137258,9.870000e-07,29801
17485,22,ZMAT5,146,30126944.0,30162969.0,620.945084,0.003213,7.477507e-14,rs737932,1.000000e-03,55954
17486,22,ZNF70,76,24083771.0,24093279.0,133.184031,0.095392,2.946103e-14,rs17629631,6.700000e-03,7621
17487,22,ZNF74,82,20748479.0,20762752.0,64.069484,0.576769,6.663133e-15,rs5749316,2.820000e-02,7625


In [22]:
sch_data.to_csv("processed_data/gwas_gene_pvals/schizophrenia/filtered_ncbi_PEGASUS_schizophrenia_gwas_data.csv", index=False)